# **Classification of Energy Consumption Levels in a Wastewater Treatment Plant**$^1$

$^1$ To be defined

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('data-melbourne.csv')

**Pre-processing**

In [3]:
df = df.drop(['VG', 'TM', 'Tm', 'VM'], axis=1)

In [5]:
df

,avg_outflow,avg_inflow,total_grid,Am,BOD,COD,TN,T,SLP,H,PP,VV,V,year,month,day
0,2.941,2.589,175856,27.0,365.0,730.0,60.378,19.3,0.0,56,1.52,10.0,26.9,2014,1,1
1,2.936,2.961,181624,25.0,370.0,740.0,60.026,17.1,0.0,63,0.00,10.0,14.4,2014,1,2
2,2.928,3.225,202016,42.0,418.0,836.0,64.522,16.8,0.0,47,0.25,10.0,31.9,2014,1,5
3,2.928,3.354,207547,36.0,430.0,850.0,63.000,14.6,0.0,49,0.00,10.0,27.0,2014,1,6
4,2.917,3.794,202824,46.0,508.0,1016.0,65.590,13.4,0.0,65,0.00,10.0,20.6,2014,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,5.068,4.716,305938,28.0,310.0,660.0,58.807,6.8,0.0,56,0.00,10.0,14.4,2019,6,20
1378,5.882,5.510,293446,40.0,250.0,700.0,60.671,7.6,0.0,71,0.00,0.0,13.5,2019,6,24
1379,4.638,4.811,307968,47.0,260.0,860.0,64.000,9.7,0.0,65,0.00,10.0,18.1,2019,6,25
1380,5.373,4.929,316675,46.0,300.0,900.0,62.479,10.9,0.0,65,0.00,10.0,21.9,2019,6,26
